# Проект 11 создание модели для определения рыночной цены авто

### Задание:

Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В вашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Вам нужно построить модель для определения стоимости. 

Заказчику важны:

- качество предсказания;
- скорость предсказания;
- время обучения.

# 1. Подготовка данных

Признаки

DateCrawled — дата скачивания анкеты из базы

VehicleType — тип автомобильного кузова

RegistrationYear — год регистрации автомобиля

Gearbox — тип коробки передач

Power — мощность (л. с.)

Model — модель автомобиля

Kilometer — пробег (км)

RegistrationMonth — месяц регистрации автомобиля

FuelType — тип топлива

Brand — марка автомобиля

NotRepaired — была машина в ремонте или нет

DateCreated — дата создания анкеты

NumberOfPictures — количество фотографий автомобиля

PostalCode — почтовый индекс владельца анкеты (пользователя)

LastSeen — дата последней активности пользователя

Целевой признак:

Price — цена (евро)

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

from sklearn import preprocessing

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder

from sklearn.metrics import mean_squared_error #RMSE для метрики

import lightgbm as lgb
from catboost import CatBoostRegressor

data = pd.read_csv('/datasets/autos.csv')

#data

#data.info()

# 2. Обучение моделей

# 2.1 Проверяем модель на библитеке LightGBM

In [2]:
#Удалим не значимые признаки
data = data.drop(['DateCrawled','RegistrationMonth','DateCreated','PostalCode','LastSeen'], axis=1)

#обработаем пропуски:
data = data.fillna('unknown')

#data = pd.get_dummies(data, drop_first=True)

#Перевод текстовых свойств в числовой формат:

columns_coding = ['VehicleType','Gearbox','Model','FuelType','Brand','NotRepaired']

label_encoder = preprocessing.LabelEncoder()
for colum in columns_coding:
    label_encoder.fit(data[colum])
    data[colum] = label_encoder.transform(data[colum]) 

df_train, df_valid_and_test = train_test_split(data, test_size=0.4, random_state=12345)
df_test, df_valid = train_test_split(df_valid_and_test, test_size=0.5, random_state=12345)

features_train = df_train.drop(['Price'], axis=1)
target_train = df_train['Price']

features_valid = df_valid.drop(['Price'], axis=1)
target_valid =  df_valid['Price']

features_test = df_test.drop(['Price'], axis=1)
target_test =  df_test['Price']

#data.info()

## Комментарий наставника
<span style="color:orange">С удалением лишних признаков полностью согласен. К слову, есть ещё один лишний признак, который можно спокойно удалить. \
Лучше не удалять пропущенные значения. Так как все они находятся в категориальных признаках, то лучшим вариантом будет заполнение их меткой «unknown»/«other». Таким образом мы сохраним значительный объем данных для обучения.</span> \
<span style="color:red">Прямое кодирование, которое ты использовал, подходит для линейных моделей (например, линейной регрессии), а для всех других моделей (включая LightGBM) более эффективно порядковое кодирование. Используй его. \
Кроме этого, разбей датасет на 3 выборки: добавь ещё тестовую к двум (для финальной оценки лучшей модели с подобранными на валидационной выборке гиперпараметрами).</span> \
<span style="color:orange">UPD 12.06.2020 Всё здорово, только для кодирования категориальных признаков **обучения** используй **OrdinalEncoder**: https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OrdinalEncoder.html?highlight=ordinalencoder#sklearn.preprocessing.OrdinalEncoder. **LabelEncoder** используется для кодирования категориальных **целевых** признаков.</span>

# Комментарий от ученика:
я прочитал как работает OrdinalEncoder и LabelEncoder, разница оказалась не такой большой, OrdinalEncoder начинает кодировать с единицы, а LabelEncoder с 0, это так значимо?

<span style="color:orange">Нет, ничего страшного не случится, если закодируешь с помощью LabelEncoder, просто OrdinalEncoder более удобен в использовании, так как позволяет закодировать сразу несколько признаков (для выполнения этой операции с LabelEncoder придётся воспользоваться циклом/функцией).</span>

In [3]:
#Отмаштабируем данные:

numeric = ['RegistrationYear', 'Power', 'Kilometer', 'NumberOfPictures']

scaler = StandardScaler()
scaler.fit(features_train[numeric])

features_train[numeric] = scaler.transform(features_train[numeric])

features_valid[numeric] = scaler.transform(features_valid[numeric])

features_test[numeric] = scaler.transform(features_test[numeric])

mean_price = data['Price'].mean()

data_train = lgb.Dataset (features_train, label = target_train)

## Комментарий наставника
<span style="color:green">Хорошо.</span> \
<span style="color:orange">UPD 13.06.2020 Признак с количеством изображений можно удалить, так как он никак не повлияет на цену авто.</span>

In [58]:
for n in range(100, 1001, 200):
    print()
    print('min_data_in_leaf =',n)
    for depth in range(1, 10, 3):
    
        params = {}
        params['max_depth'] = depth
        params['num_leaves'] = 2**depth
        params['min_data_in_leaf'] = n
        clf = lgb.train (params, data_train, 100)

        predictions = clf.predict(features_valid)
        rmse = mean_squared_error(predictions, target_valid)**0.5
        
        print()
        print('rmse =',rmse)
        
        print()
        print('depth =',depth,'Модель в среднем ошибается в',rmse/mean_price,'раз от средней цены')
        #print()


min_data_in_leaf = 100

rmse = 2621.232863545899

depth = 1 Модель в среднем ошибается в 0.5934880151612925 раз от средней цены

rmse = 1975.901239281225

depth = 4 Модель в среднем ошибается в 0.4473748673627593 раз от средней цены

rmse = 1836.1781782487453

depth = 7 Модель в среднем ошибается в 0.41573938647219455 раз от средней цены

min_data_in_leaf = 300

rmse = 2621.232863545899

depth = 1 Модель в среднем ошибается в 0.5934880151612925 раз от средней цены

rmse = 1980.319554324417

depth = 4 Модель в среднем ошибается в 0.4483752428203575 раз от средней цены

rmse = 1858.0083504034365

depth = 7 Модель в среднем ошибается в 0.4206820780288656 раз от средней цены

min_data_in_leaf = 500

rmse = 2621.232863545899

depth = 1 Модель в среднем ошибается в 0.5934880151612925 раз от средней цены

rmse = 1989.2297619830824

depth = 4 Модель в среднем ошибается в 0.45039265284583035 раз от средней цены

rmse = 1877.4988365869367

depth = 7 Модель в среднем ошибается в 0.42509502817932

# 2.1.1 Анализ модели

Методом перебора, был выбран гиперпараметр max_depth 16, который даёт максимальное качество модели, 

дальшейнее увеличение гиперпараметра ведёт не ведёт к увеличению качества модели, но увеличивает время обучения.

In [4]:
%%time
# cчитаем время с параметром ['min_data_in_leaf'] = 50

params = {}
max_depth = 16
params['max_depth'] = max_depth
params['num_leaves'] = 2**max_depth
params['min_data_in_leaf'] = 50
clf = lgb.train (params, data_train, 100)

predictions = clf.predict(features_valid)
rmse = mean_squared_error(predictions, target_valid)**0.5

print('rmse =',rmse)

print()
print('depth =',max_depth,'Модель в среднем ошибается в',rmse/mean_price,'раз от средней цены')
print()

rmse = 1729.7911857161218

depth = 16 Модель в среднем ошибается в 0.39165171157872725 раз от средней цены

CPU times: user 4min 55s, sys: 4.13 s, total: 5min
Wall time: 5min 3s


In [5]:
%%time
# cчитаем время с параметром ['min_data_in_leaf'] = 1000

params = {}
max_depth = 16
params['max_depth'] = max_depth
params['num_leaves'] = 2**max_depth
params['min_data_in_leaf'] = 1000
clf = lgb.train (params, data_train, 100)

predictions = clf.predict(features_valid)
rmse = mean_squared_error(predictions, target_valid)**0.5

print('rmse =',rmse)

print()
print('depth =',max_depth,'Модель в среднем ошибается в',rmse/mean_price,'раз от средней цены')
print()

rmse = 1867.6869600990362

depth = 16 Модель в среднем ошибается в 0.42287346626363387 раз от средней цены

CPU times: user 24.8 s, sys: 147 ms, total: 24.9 s
Wall time: 25.2 s


У второй модели по сравнению со первой:

Качество ухудшилось на 8%, зато скорость возрасла в более чем 6,5 раз.

Меняя гиперпараметр 'min_data_in_leaf' - можно увеличить скорость обучения, но так же ухудшается качество обучения.

# 2.1.2 Проверка модели на тестовых данных:

In [6]:
predictions = clf.predict(features_test)
rmse = mean_squared_error(predictions, target_test)**0.5

print(rmse)
print()
print('Модель в среднем ошибается в',rmse/mean_price,'раз от средней цены')

1854.267897264694

Модель в среднем ошибается в 0.41983518108201706 раз от средней цены


# 2.2 Проверяем модель на библиотеке CatBoost

In [7]:
data = pd.read_csv('/datasets/autos.csv')

#Удалим не значимые признаки
data = data.drop(['DateCrawled','RegistrationMonth','DateCreated','PostalCode','LastSeen'], axis=1)

#обработаем пропуски:
data = data.fillna('unknown')

#Перевод текстовых свойств в числовой формат:

columns_coding = ['VehicleType','Gearbox','Model','FuelType','Brand','NotRepaired']

#label_encoder = preprocessing.LabelEncoder()
#for colum in columns_coding:
#    label_encoder.fit(data[colum])
#    data[colum] = label_encoder.transform(data[colum]) 

df_train, df_valid_and_test = train_test_split(data, test_size=0.4, random_state=12345)
df_test, df_valid = train_test_split(df_valid_and_test, test_size=0.5, random_state=12345)

features_train = df_train.drop(['Price'], axis=1)
target_train = df_train['Price']

features_valid = df_valid.drop(['Price'], axis=1)
target_valid =  df_valid['Price']

features_test = df_test.drop(['Price'], axis=1)
target_test =  df_test['Price']

In [8]:
%%time

cat_features = list(features_train.columns)

#cat_features = ['VehicleType', 'RegistrationYear', 'Gearbox', 'Power', 'Model', 'Kilometer', 'FuelType', 'Brand', 'NotRepaired', 'NumberOfPictures']

model = CatBoostRegressor(iterations=100)

model.fit(features_train, target_train, cat_features=cat_features, verbose=25)

probabilities_valid = model.predict(features_valid)
#probabilities_one_valid = probabilities_valid[:, 1]

rmse = mean_squared_error(target_valid, probabilities_valid)**0.5

print(rmse)
print()
print('Модель в среднем ошибается в',rmse/mean_price,'раз от средней цены')

0:	learn: 4433.9533741	total: 558ms	remaining: 55.2s
25:	learn: 3207.8570487	total: 10.1s	remaining: 28.9s
50:	learn: 2651.2510410	total: 19.4s	remaining: 18.7s
75:	learn: 2375.4155111	total: 28.8s	remaining: 9.1s
99:	learn: 2262.6992779	total: 37.9s	remaining: 0us
2273.7885636728925

Модель в среднем ошибается в 0.5148212050588924 раз от средней цены
CPU times: user 37.2 s, sys: 2.47 s, total: 39.6 s
Wall time: 43.1 s


## Комментарий наставника
<span style="color:orange">UPD 13.06.2020 Ты указал все признаки как категориальные, а не только действительно категориальные.</span>

# проверка на тестовых данных:

In [9]:
probabilities_valid = model.predict(features_test)
#probabilities_one_valid = probabilities_valid[:, 1]

rmse = mean_squared_error(target_test, probabilities_valid)**0.5

print(rmse)
print()
print('Модель в среднем ошибается в',rmse/mean_price,'раз от средней цены')

2250.774802587906

Модель в среднем ошибается в 0.5096105305027798 раз от средней цены


# Вывод:
конкретно с этой задачью, отлично справились обе библиотеки: CatBoost и LightGBM

обе библиотеки имеют свои минусы и плюсы, выбор лучшей библиотеки - это сложный вопрос, для точного ответа на который потребуется протестировать библиотеки на различных задачах, а после сделать вывод, какая лучше.

если прочитать информацию по ссылке https://catboost.ai/#benchmark

CatBoost - намного быстрее, и чуть более точен, чем LightGBM. 

Ура, замечательным разработчикам из Яндекса! <span style="color:green">:)</span>